<a href="https://colab.research.google.com/github/ABDA48/LangchainNotebook/blob/main/Build_an_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [24]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
os.environ['TAVILY_API_KEY']=userdata.get('TAVILY_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro")

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

# **End-to-end agent**
The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the in

In [6]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [8]:
!pip install langchain-core langgraph>0.2.27

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# **Tavily**
We will be using Tavily (a search engine) as a tool. In order to use it, you will need to get and set an API key:
We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.


In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1731180333, 'localtime': '2024-11-09 11:25'}, 'current': {'last_updated_epoch': 1731179700, 'last_updated': '2024-11-09 11:15', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 136, 'wind_dir': 'SE', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 38, 'cloud': 50, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'windchill_c': 14.5, 'windchill_f': 58.0, 'heatindex_c': 14.4, 'heatindex_f': 58.0, 'dewpoint_c': 8.9, 'dewpoint_f': 48.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.6, 'gust_mph': 6.7, 'gust_kph': 10.8}}"}, {'url': 'https://forecast.weather.gov/MapClick

In [25]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.7 MB/s eta 0:00:00


In [26]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [27]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.

In [30]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi!
ToolCalls: []


In [32]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_cq8y', 'type': 'tool_call'}]


# **Create the agent**
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

In [33]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [35]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='271c02aa-f3c4-4e93-8cca-73414ae206d8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vps6', 'function': {'arguments': '{"query":"hi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 941, 'total_tokens': 1011, 'completion_time': 0.058333333, 'prompt_time': 0.123620349, 'queue_time': 0.03267116499999999, 'total_time': 0.181953682}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-03f657da-2c15-4ede-ae16-5a193ca115fa-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hi'}, 'id': 'call_vps6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 941, 'output_tokens': 70, 'total_tokens': 1011}),
 ToolMessage(content="HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')

In [39]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Madagascar?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Madagascar?', additional_kwargs={}, response_metadata={}, id='0971bd41-f106-4d36-9c7f-0741a7d50349'),
 AIMessage(content="Madagascar's weather varies greatly depending on the season and region. The country is located near the equator and has a tropical climate, with two main seasons: a dry season and a rainy season.\n\nDuring the dry season, which typically runs from May to September, the weather is generally sunny and dry, with average temperatures ranging from 65°F to 85°F (18°C to 30°C).\n\nDuring the rainy season, which typically runs from October to April, the weather is often cloudy and rainy, with average temperatures ranging from 60°F to 80°F (15°C to 27°C).\n\nMadagascar's coastal regions tend to be cooler than the inland areas, with average temperatures ranging from 50°F to 70°F (10°C to 21°C).\n\nIt's worth noting that Madagascar is a large and diverse country, with different regions having different climates and weather patterns."

In [45]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_key5', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1913, 'total_tokens': 1986, 'completion_time': 0.060833333, 'prompt_time': 0.222314378, 'queue_time': -0.335879767, 'total_time': 0.283147711}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-785b2bfc-a88a-4088-adec-981a67e58d0e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_key5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1913, 'output_tokens': 73, 'total_tokens': 1986})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\

In [46]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [47]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [48]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4fwq', 'function': {'arguments': '{"query":"hi im bob"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1905, 'total_tokens': 1977, 'completion_time': 0.06, 'prompt_time': 0.268472066, 'queue_time': -0.347957878, 'total_time': 0.328472066}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4391d64a-bd46-499a-84cd-2d406fc7e59e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hi im bob'}, 'id': 'call_4fwq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1905, 'output_tokens': 72, 'total_tokens': 1977})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.allthelyrics.com/lyrics/disney-hi_im_bob", "content": "Hi I\'m bob I\'m a goldfish And a goldfish has a memory of a ... H

In [57]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the president off united state")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='As of my knowledge cutoff in 2021, the President of the United States is Joe Biden.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 6010, 'total_tokens': 6031, 'completion_time': 0.019110292, 'prompt_time': 0.280259449, 'queue_time': 0.0032485260000000293, 'total_time': 0.299369741}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-b4295dc2-383e-491c-9bc7-0443cda945cf-0', usage_metadata={'input_tokens': 6010, 'output_tokens': 21, 'total_tokens': 6031})]}}
----


In [50]:
#If you want to start a new conversation, all you have to do is change the thread_id used
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm not sure what your name is, as I don't have any information about you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 1908, 'total_tokens': 1928, 'completion_time': 0.016666667, 'prompt_time': 0.226638625, 'queue_time': -0.298119648, 'total_time': 0.243305292}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-42c5ada1-2fd6-471e-83f1-8c672fa095f9-0', usage_metadata={'input_tokens': 1908, 'output_tokens': 20, 'total_tokens': 1928})]}}
----
